<a href="https://colab.research.google.com/github/francji1/01ZLMA/blob/main/R/01ZLMA_ex02_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01ZLMA - Exercise 02
Exercise 02 of the course 01ZLMA. 

## Contents

* Fast and furious necessary theory recap from Lectures 01-03
 ---
* Numeric methods of MLE calculations 
 ---
* Newton-Raphson
* Fiher scoring - IWLS
* SGD
* ..



#  Necessary theory recap from Lectures 01-03

Let's consider (m1):
  1. realization $(y_i,\ldots,y_n)$ of $iid$ random variables $Y_1,\ldots,Y_n$ with probability density function $f(y;\theta;\phi)$ from an exponential family of probability distributions
  $$f(y;\theta;\phi) = exp(\frac{y \theta - b(\theta)}{a(\phi)} - c(y,\phi)),$$
  where conditions of regularity are fulfilled (one dimensional case, i.e. $y_i,\theta_i \in R)$.
  2. Regression matrix $X$ and vector of unknown parameters $\beta$, linear predictor $η = X \beta$
  3. A link function $g(x)$
  $$\eta_i = g(\mu_i) = x_i^T \beta, \ \text{where} \ \mu_i = E[Y_i] \ \ i = 1,\ldots,n$$

The dispersion $a(\phi)$ is typically known. If not, we take it as nuisance parameter.

Link function satisfying $g(\mu_i) = \theta_i$ is called canonical.

For $b(\theta) \in C^2$ we showed:
$$E[Y] = b'(\theta) $$
$$V[Y] = a(\phi) b''(\theta) $$
and defined variance function $v(\mu) = \frac{\partial \mu}{\partial \theta}$, i.e. $V[Y] = a(\phi) v(\mu)$

Relations:

$$
\beta \xrightarrow[]{\eta_i = x_i^T\beta} \eta
\xrightarrow[]{\mu_i = g^{-1}(\eta_i)}  \mu
\xrightarrow[]{\theta_i = (b')^{-1}(\mu_i)}  \theta
$$ 

Inverse relatiions
$$
\eta_i 
\xleftarrow[]{}  \mu
\xleftarrow[]{}  \theta
$$ 


## Exercise 0

Compute $E[Y]$, $V[Y]$, and $v(Y)$ by the help of moment generating function theory for the following disributions:
*  Normal: $N(\mu,\sigma^2)$
*  Gamma: $\Gamma[a,p]$
* Inverse: Gaussian $IG[\mu, \lambda]$
* Poisson: $Poi(\lambda)$
* Bernoulli: $Ber(p)$

Questions:
* Which distributions can fulfill homoscedasticity?
* For which distribution the variance increases with the square of the mean value?
* Does exists a distribution, where $v(Y) = k \cdot \mu$ ?


## Likelihood, Score function, Information matrix

* Likelihood function:
$$L_n(\Theta) = L_n(\Theta|Y) = \prod_{i=1}^{n} f(y_i|\Theta) $$
* log-likelihood function:
$$l_n(\Theta) = l_n(\Theta|Y) = \sum_{i=1}^{n} \text{ln} f(y_i|\Theta) $$
* Score function:  ($R^m \rightarrow R^m$):
$$U = U(\Theta|Y_i) = \frac{\partial \text{ln} f(y_i|\Theta)}{\partial \Theta} $$
* Score vector (statistics):
$$U_{n} = U_{n}(\Theta|Y) = \sum_{i=1}^{n} U(\Theta|Y_i)= \sum_{i=1}^{n} \frac{\partial l_i(\Theta|y_i)}{\partial \Theta} $$
* Fisher Information matrix
$$ I_n (\Theta) = E_{\theta}[U_nU_n^T]$$
with elements
$$ I_{n,j,k} = E_{\theta}[\frac{\partial l}{\partial \theta_j}\frac{\partial l}{\partial \theta_k}] = -E_{\theta}[\frac{\partial^2 l}{\theta_j \theta_k}]$$


Question:
Why is the second derivative called information?



In [ ]:
library(tidyverse)
library(lubridate)
library(MASS)

#install.packages("car")
#library(car)
#install.packages("GGally")
#library(GGally)


#For sure: set dplyr functions
select    <- dplyr::select;
rename    <- dplyr::rename;
mutate    <- dplyr::mutate; 
summarize <- dplyr::summarize;
arrange   <- dplyr::arrange;
slice     <- dplyr::slice;
filter    <- dplyr::filter;
recode    <- dplyr::recode



In [ ]:
data()

Datasets we will use:

Peter K. Dunn • Gordon K. Smyth, Generalized Linear ModelsWith Examples in R

https://link.springer.com/content/pdf/10.1007%2F978-1-4419-0118-7.pdf

In [ ]:
install.packages("GLMsData")
library(GLMsData)
#install.packages("splines")
#library(splines)
#install.packages("statmod")
#library(statmod)
#install.packages("tweedie")
#library(tweedie)

A. J. Dobson AN INTRODUCTION TO GENERALIZED LINEAR MODELS


https://link.springer.com/content/pdf/10.https://reneues.files.wordpress.com/2010/01/an-introduction-to-generalized-linear-models-second-edition-dobson.pdf

In [ ]:
install.packages("dobson")
library(dobson)

CRAN packages for generalized linear models and with related methods

https://cran.r-project.org/web/packages/cranly/vignettes/glms.html



In [ ]:
data()

Exercise 01

Estimate parameters $\beta$ by MLE for M1.

log-likelihood function is
$$ l(\theta, \phi, y) = \sum_{i=1}^n \frac{y_i \theta_i - b(\theta_i)}{a_i(\phi)} + \sum_{i=1}^n c(y,\phi) $$
and we want to estimate $\beta = (\beta_1, \ldots, \beta_n)^T$, i.e.
$$ \hat{\beta} = argmax_{\beta}(l(\theta,\phi,y))$$
$$\Rightarrow$$
$$ U_n = \sum_{i=1}^n \frac{y_i - \mu_i}{V[Y_i]  g'(\mu_i)} x_i = X^T M^{-1}(y-\mu) = 0$$
$$\Rightarrow$$
$$ U_n = X^T W(\beta)^{-1}Z(\beta) = 0,$$\
where $W(\beta) = diag(V[Y_i]g'(\mu_i)^2)$ and $Z(\beta) = diag(g'(\mu_i))(y-\mu)$


It can be shown 
$$I_n = X^T W(\beta)^{-1} X $$

## Newton Rapson

For MLE using the score function, the estimating equation is
$$\hat{\beta}^{(r+1)} = \hat{\beta}^{(r)} + \frac{U(\hat{\beta}^{(r)})}{\frac{d U(\hat{\beta}^{(r)})}{d \theta}}$$

Question: What are advantages and disadvantages of this method.

## Fisher Scoring

$$\hat{\beta}^{(r+1)} =  \hat{\beta}^{(r)} + \frac{U(\hat{\beta}^{(r)})}{E [\frac{d U(\hat{\beta}^{(r)})}{d \theta} ]} = \hat{\beta}^{(r)} + \frac{U(\hat{\beta}^{(r)})}{I(\hat{\beta}^{(r)})}$$

Question: What are advantages and disadvantages of this method.

# IWLS

$$I(\hat{\beta}^{(r)}) \hat{\beta}^{(r+1)}  =  I(\hat{\beta}^{(r)}) \hat{\beta}^{(r)} + U(\hat{\beta}^{(r)})$$
$$ \Rightarrow$$
$$(X^T W(\hat{\beta}^{(r)})^{-1} X) \hat{\beta}^{(r+1)}  = X^T W(\hat{\beta}^{(r)})^{-1} Z(\hat{\beta}^{(r)})  $$
$$ \Rightarrow$$
$$  \hat{\beta}^{(r+1)}  =  (X^T W(\hat{\beta}^{(r)})^{-1} X)^{-1} X^T W(\hat{\beta}^{(r)})^{-1} Z(\hat{\beta}^{(r)})  $$ 
where $Z(\beta) = X \beta + diag(g'(\mu_i))(y-\mu)$



## Poisson regression example (Dobson 4.4)



In [ ]:
# Generate and plot the Dataset

X <- c(-1,-1,0,0,0,0,1,1,1,2,2,2,2)
Y <- c(2,3,6,7,8,9,10,12,15,16,23,20,25)
n <- length(X)

df1 <- data.frame(X=X, Y=Y)
plot(X,Y,xlim = c(-1.5, 2.5), ylim = c(0, 30), col="red",lwd=6)

In [ ]:
install.packages("plotly")
library(plotly)
plot_ly(df1, x=~ X, y=~Y, mode = "markers") %>%
 layout(title = "Scatter Plot by plotly") %>%
embed_notebook()

Let us assume that the response $Y_i$ are Poisson random variables and model 

the relation ship between $Y_i$ and $x_i$ by the straight line, i.e.
$$E[Y_i] = \mu_i = \beta_1 + \beta_2 x_i = x_i^T \beta \  \Rightarrow \ q(\mu_i) = \mu_i =  x_i^T \beta  = \eta_i$$
Therefore $\frac{1}{g'(\mu_i)} = 1 \ \Rightarrow \ w_{ii} = \frac{1}{V[Y]} = \frac{1}{\beta_1 + \beta_2x_i}$ 

In [ ]:
# function to calcualate weights
calc1_W <- function(X,beta){
  n=length(X[,1])
w = diag(c(1/(X%*%beta)),n,n)
}

In [ ]:
# Initial values
X=cbind(rep(1,length(n)),c(-1,-1,0,0,0,0,1,1,1,2,2,2,2))
beta_0 = c(7,5)
z = Y

In [ ]:
# Compute Weight matrix
W <- calc1_W(X,beta_0)
W

In [ ]:
# Step from beta_0 to beta_1
beta_1 = solve(t(X)%*%W%*%X, t(X)%*%W%*%z)
beta_1

In [ ]:
# variance-covariance matrix for estamtes beta_1
W = calc1_W(X,beta_1)
solve((t(X)%*%W%*%X))

In [ ]:
#Write function

# Your turn:
1. Write function to calculate IWLS for example 1 with following parameters
 * maximal number of iteration
 * accuracy
 * initial estimation
2. Try different initialization (ols, random, lecture, ...). Plot convergence of parameters $\beta_i$ depends on initial estimation.


In [ ]:
beta_init = lm(Y~-1+X)$coeff
beta_init

In [ ]:
# Set initial parameters
maxiter <- 20        # maximal number of iteration
epsilon <- 10^(-6)   # accuracy
#beta_init <- c(10,5)
beta_all <- rbind(beta_init)
beta_all

In [ ]:
  W = calc1_W(X,beta_init)
  diag(W)

In [ ]:
IWLS_1 <- function(X,Y,beta_init,maxiter,epsilon){
  # Fisher-scoring algorithm
  i <- 1     # first iteration
  convergence <- FALSE
  beta_i <- beta_init
  while (convergence == FALSE & i <= maxiter)
  {
  ## Place for your code ###
  W = calc1_W(X,beta_i)
  beta_i = solve(t(X)%*%W%*%X, t(X)%*%W%*%Y)
  print(i)
  print(beta_i)
  print(beta_all)
  diff   <- max(abs(t(beta_i) - beta_all[i,]))
  print(diff)
  if (any(diff>=epsilon)){
     i <- i+1
     beta_all <- rbind(beta_all,t(beta_i))
  }
  else
     convergence <- TRUE
  }
  return(data.frame(beta_all))
}

In [ ]:
betas <- IWLS_1(X,Y,beta_init,maxiter,epsilon)
names(betas) <-c("beta0","beta1")
betas <- betas %>% mutate(iteration = c(1:dim(betas)[1]))
betas

In [ ]:
p = ggplot() + 
  geom_line(data = betas, aes(x = iteration, y = beta0), color = "blue") +
  geom_line(data = betas, aes(x = iteration, y = beta1), color = "red") +
  xlab('iteration') +
  ylab('coeff value')
print(p)

In [ ]:
#############################################
## By default R function

model <- glm(Y~-1+X, family=poisson(link = "identity"))

summary(model) 
#beta_hat <- coefficients(model)  
#beta_hat

# Example 2


An example 4.1. from the book "An Introduction to Generalized Linear Models" by Dobson. The data in Table 4.5 show the numbers of cases of AIDS in Australia by date of diagnosis for successive 3-months periods from 1984 to 1988.

* Plot the number of cases $y_i$ against time period (i = 1 ... 20).
* Use the Poisson distribution with parameter $\lambda_i = i^{\theta}$ or equivalently $log(\lambda_i) = \theta log(i)$. Plot $log(y_i)$ against $log(i)$ to examne this model.
* Fit a generalized linear model to these data using the Poisson distribution and the log-link function, ie 
$$ g(\lambda_i) = log(\lambda_i) = \beta_1 + \beta_2 x_i,$$
where $x_i = log_i$.

In [ ]:
data(aids)
summary(aids)
aids

In [ ]:
AIDS <- aids %>%
  mutate(season = paste0(year,":Q",quarter),
         time = yq(season))
head(AIDS)

In [ ]:
p <- ggplot(AIDS, aes(x=time, y=cases)) +
  geom_line() + 
  #scale_x_date(date_labels = "%Y-%B")#+
  theme(axis.text.x=element_text(angle=60, hjust=1)) +
  scale_x_date(breaks = AIDS$time)
p

In [ ]:
X <- seq(1,20,1)
Y <- aids$cases

par(mfrow=c(1,3))
plot(X,Y, col="red", lwd=6, main="Obr. 1")
plot(log(X),Y, col="red",lwd=6, main="Obr. 2 - log X")
plot(log(X),log(Y),col="red",lwd=6, main="Obr. 3 - log X, log Y")


In [ ]:
install.packages("plotly")
library(plotly)
#X <- seq(1,20,1)
#Y <- c(1, 6, 16, 23, 27, 39, 31, 30, 43, 51, 63, 70, 88, 97, 91, 104, 110, 113, 149, 159)

df2 <- data.frame(x = c(X,log(X),log(X)),
                  y = c(Y,Y,log(Y)),
                  group = rep(c("X vs. Y","log(X) vs. Y","log(X) vs. log(Y)"), each = 20))

pl <- plot_ly(df2, x=~ x, y=~y, mode = "markers",split = ~group) %>%
  layout(title = "Scatter Plot by plotly")
embed_notebook(pl)

## Your turn:
Write function to estimate coefficients by IWLS in Example 2 

Estimate $W^{-1}(\beta)$

$$W^{-1}(\beta) = diag(\frac{1}{V[Y_i]g'(\mu_i)^2)} = diag(\frac{\lambda_i^2}{\lambda_i}) = diag(e^{\beta_0 + \beta_1 x_i })$$


In [ ]:
# function to calcualate weights
calc2_W_inv <- function(X,beta){
  n = length(X[,1])
  W = diag(c(exp(X%*%beta)),n,n)
  return(W)
}

Estimate $Z_i(\beta)$

$$Z_i(\beta) = X \beta + diag(g'(\mu_i))(y-\mu) = X \beta + \frac{y -e^{\beta_0 + \beta_1 x_i}}{e^{\beta_0 + \beta_1 x_i}}$$


In [ ]:
# function to calcualate weights
calc2_Z <- function(X,Y,beta){
  Z = X%*%(beta) + (Y-exp(X%*%beta))/exp(X%*%beta)
  return(Z)
 }

In [ ]:
Y <- aids$cases
n <- length(Y) 
X <- cbind(rep(1,n), log(seq(1,n,1)))
#cbind(Y,X)

In [ ]:
### Initial values ###
beta_init      <- lm(log(Y)~ -1+ X)$coeff
dim(beta_init) <- c(2,1) # column vector
#print(beta_init)

Z_1<- calc2_Z(X,Y,beta_init)
#print(z_1)

# Compute Weight matrix
W_1 <- calc2_W_inv(X,beta_init)
#diag(W_1)

In [ ]:
# Step from beta_0 to beta_1 and 
beta_1 = solve(t(X)%*%W_1%*%X, t(X)%*%W_1%*%Z_1)
beta_1
# variance-covariance matrix for estamtes beta_1
W_2 = calc2_W_inv(X,beta_1)
solve((t(X)%*%W_2%*%X))

In [ ]:
# IWLS for example 2
IWLS_2 <- function(X,Y,beta_init,maxiter,epsilon){
  # Fisher-scoring algorithm
  i <- 1     # first iteration
  convergence <- F
  beta_all <- matrix(NA, nrow=maxiter, ncol=length(beta_init))
  beta_all[1,] <- t(beta_init)
  beta_i <- beta_init

  while (convergence == F & i <= maxiter){
    W = calc2_W_inv(X,beta_i)
    Z = calc2_Z(X,Y,beta_i)
    beta_i = solve(t(X)%*%W%*%X, t(X)%*%W%*%Z)
    #print(i)
    #print(t(X)%*%W%*%X)
    #print(t(beta_i))
    diff   <- max(abs(t(beta_i) - beta_all[i,]))
    #print(diff)
    if (diff>=epsilon){
      i <- i+1
      beta_all[i,] <- t(beta_i)
    }
    else
      convergence <- T
    }
  return(data.frame(beta_all) %>% na.omit)
}

In [ ]:
# Initialization
maxiter = 100
epsilon = 10^(-8)

# Estimation of betas
betas <- IWLS_2(X,Y,beta_init,maxiter,epsilon)
names(betas) <-c("beta0","beta1")
betas

In [ ]:
betas <- betas %>% mutate(iteration = c(1:dim(betas)[1]))
p = ggplot() + 
  geom_line(data = betas, aes(x = iteration, y = beta0), color = "blue") +
  geom_line(data = betas, aes(x = iteration, y = beta1), color = "red") +
  xlab('iteration') +
  ylab('coeff value')
print(p)

In [ ]:
# Use beta_init = c(1,1)
betas <- IWLS_2(X,Y,c(0.1,0.1),maxiter,epsilon)
names(betas) <-c("beta0","beta1")
betas

In [ ]:
betas <- betas %>% mutate(iteration = c(1:dim(betas)[1]))
p = ggplot() + 
  geom_line(data = betas, aes(x = iteration, y = beta0), color = "blue") +
  geom_line(data = betas, aes(x = iteration, y = beta1), color = "red") +
  xlab('iteration') +
  ylab('coeff value')
print(p)

## Solution from the lecture. 

Direct computation of $U(\beta)$ and $I(\beta)$

In [ ]:
# TH solution from the lecture
FishScor2 <- function(x,Y,b){
  result <- list(FM=NA, SV=NA)
  pom <- exp(b[2]*x)
  
  a11 <- exp(b[1])*sum(pom)
  a12 <- exp(b[1])*sum(x*pom)
  a22 <- exp(b[1])*sum(x^2*pom)
  
  # Information matrix
  result$FM <- matrix(data=c(a11,a12,a12,a22), nrow = 2)
  
  # Score vector
  u1 <- sum(Y) - exp(b[1])*sum(pom)
  u2 <- sum(x*Y) - exp(b[1])*sum(x*pom)
  
  result$SV <- matrix(data=c(u1,u2), nrow = 2)
  return(result)
}


In [ ]:
# Initialization
X2 <- seq(1,20,1)
Y2 <- c(1, 6, 16, 23, 27, 39, 31, 30, 43, 51, 63, 70, 88, 97, 91, 104, 110, 113, 149, 159)
maxiter <- 20         # maximalni pocet iteraci
epsilon <- 10^(-6)    # presnost

i <- 1     # aktualni iterace
beta <- matrix(0,2,maxiter)
beta[,1] <- beta_init
convergence <- F

# Fisher-scoring algoritmus
while (convergence == F & i <= maxiter){
  FS <- FishScor2(log(X2),Y2,beta[,i])
  # print(FS$FM) # Information matrix
  
  beta[,i+1] <- beta[,i] + solve(FS$FM)%*%FS$SV
  
  diff <- (abs(beta[,i+1] - beta[,i]))
  if (any(diff>=epsilon)){
    i <- i+1
  }
  else
    convergence <- TRUE
}
beta.est <- beta[,i] 
beta.est


In [ ]:
#############################################
## Use glm() function: Built-in function from R

model <- glm(formula=Y~log(X2),family=poisson(link = "log"))
summary(model) # souhrn modelu
beta.e <- coefficients(model); beta.e  # odhadnute parametry
y.hat  <- model$fitted.values

#############################################
## Plot data and estimation
plot(X2,Y, col="red", cex=1.5, lwd=2, 
      main="Poissonův model",
      xlab="číslo čtvrtletí",
      ylab="Počet nových případů",
      cex.lab=1.2)
lines(y.hat, col="blue", type = "l", lwd=3)
text <- c("model", "data")
legend(x=1, y=160, text, col = c(4,2), bty="n", lwd = 2, pch=c(26,1), cex=1.3, lty=c(1,0))
dev.off()


## Logistic regression and IWLS

http://www.jtrive.com/estimating-logistic-regression-coefficents-from-scratch-r-version.html